In [13]:
import os
import json
import gc
from pathlib import Path
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
from transformers import PreTrainedTokenizerFast
from tokenizers import (
    decoders,
    models,
    normalizers,
    pre_tokenizers,
    processors,
    trainers,
    Tokenizer,
)
from typing import Dict, List, Set, Tuple, NamedTuple, Callable, Any
import textstat
import scml
from scml import pandasx as pdx
from daigt.preprocess import en as pen
tim = scml.Timer()
tim.start()
os.environ["TOKENIZERS_PARALLELISM"] = "false"
percentiles=[.01, .05, .1, .2, .3, .4, .5, .6, .7, .8, .9, .95, .99]
pd.set_option("use_inf_as_na", True)
pd.set_option("max_info_columns", 9999)
pd.set_option("display.max_columns", 9999)
pd.set_option("display.max_rows", 9999)
pd.set_option('max_colwidth', 9999)
tqdm.pandas()
scml.seed_everything()
info = np.iinfo(np.int16)
print(f"int16, min={info.min}, max={info.max}")

int16, min=-32768, max=32767


In [2]:
text_col = "text_bsc"

In [3]:
df = pd.read_parquet("input/preprocess.parquet")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39120 entries, 0 to 39119
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   essay_id        39120 non-null  int32  
 1   generated       39120 non-null  int8   
 2   source          39120 non-null  object 
 3   prompt          39120 non-null  object 
 4   text            39120 non-null  object 
 5   text_bsc        39120 non-null  object 
 6   text_bow        39120 non-null  object 
 7   text_bow_len    39120 non-null  int16  
 8   prompt_bsc      39120 non-null  object 
 9   prompt_bow      39120 non-null  object 
 10  prompt_bow_len  39120 non-null  int16  
 11  white_sim       39120 non-null  float32
dtypes: float32(1), int16(2), int32(1), int8(1), object(7)
memory usage: 2.6+ MB


# Character level features

In [4]:
%%time
col = "ch_len"
df[col] = df[text_col].str.len()
df[col] = df[col].astype(np.int32)

def digit_frac(row) -> float:
    return pen.digit_frac(row[text_col])


def letter_frac(row) -> float:
    return pen.letter_frac(row[text_col])


def space_frac(row) -> float:
    return pen.space_frac(row[text_col])


def punc_frac(row) -> float:
    return pen.punc_frac(row[text_col])


def upper_frac(row) -> float:
    return pen.upper_frac(row[text_col])


def repeat_char_frac(row) -> float:
    return pen.repeat_char_frac(row[text_col])


def repeat_substring_frac(row) -> float:
    return pen.repeat_substring_frac(row[text_col])


char_fns: Dict[str, Callable] = {
    "ch_digit_frac": digit_frac,
    "ch_letter_frac": letter_frac,
    "ch_space_frac": space_frac,
    "ch_punc_frac": punc_frac,
    "ch_upper_frac": upper_frac,
    "ch_repeat_char_frac": repeat_char_frac,
    #"ch_repeat_substring_frac": repeat_substring_frac,
}

for col, fn in char_fns.items():
    print(col)
    df[col] = df.progress_apply(fn, axis=1)
    df[col] = df[col].astype(np.float32)

ch_digit_frac


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 39120/39120 [00:01<00:00, 22171.96it/s]


ch_letter_frac


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 39120/39120 [00:01<00:00, 20050.18it/s]


ch_space_frac


...

ch_upper_frac


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 39120/39120 [00:01<00:00, 22133.53it/s]


ch_repeat_char_frac


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 39120/39120 [00:05<00:00, 7319.50it/s]

CPU times: user 14.4 s, sys: 79.7 ms, total: 14.5 s
Wall time: 14.5 s


# Textstat Features

In [5]:
def syllable_count(row) -> int:
    return textstat.syllable_count(row[text_col])


def lexicon_count(row) -> int:
    return textstat.lexicon_count(row[text_col])


def sentence_count(row) -> int:
    return textstat.sentence_count(row[text_col])


def syllables_per_word(row) -> float:
    return row["ts_syllable_count"] / (row["ts_lexicon_count"] + 1)


def syllables_per_sent(row) -> float:
    return row["ts_syllable_count"] / (row["ts_sentence_count"] + 1)


def words_per_sent(row) -> float:
    return row["ts_lexicon_count"] / (row["ts_sentence_count"] + 1)
    

def polysyllable_frac(row) -> float:
    return textstat.polysyllabcount(row[text_col]) / (row["ts_lexicon_count"] + 1)


def monosyllable_frac(row) -> float:
    return textstat.monosyllabcount(row[text_col]) / (row["ts_lexicon_count"] + 1)


def flesch_reading_ease(row) -> float:
    return textstat.flesch_reading_ease(row[text_col])


def flesch_kincaid_grade(row) -> float:
    return textstat.flesch_kincaid_grade(row[text_col])


def gunning_fog(row) -> float:
    return textstat.gunning_fog(row[text_col])


def smog_index(row) -> float:
    return textstat.smog_index(row[text_col])


def automated_readability_index(row) -> float:
    return textstat.automated_readability_index(row[text_col])


def coleman_liau_index(row) -> float:
    return textstat.coleman_liau_index(row[text_col])


def linsear_write_formula(row) -> float:
    return textstat.linsear_write_formula(row[text_col])


def dale_chall_readability_score(row) -> float:
    return textstat.dale_chall_readability_score(row[text_col])


def difficult_words(row) -> float:
    return textstat.difficult_words(row[text_col])


def spache_readability(row) -> float:
    return textstat.spache_readability(row[text_col])


def mcalpine_eflaw(row) -> float:
    return textstat.mcalpine_eflaw(row[text_col])


stage1: List[Tuple[str, Callable, Any]] = [
    ("ts_syllable_count", syllable_count, np.int32),
    ("ts_lexicon_count", lexicon_count, np.int32),
    ("ts_sentence_count", sentence_count, np.int32),
]
stage2: List[Tuple[str, Callable, Any]] = [
    ("ts_syllables_per_word", syllables_per_word, np.float32),
    ("ts_syllables_per_sent", syllables_per_sent, np.float32),
    ("ts_words_per_sent", words_per_sent, np.float32),
    ("ts_polysyllable_frac", polysyllable_frac, np.float32),
    ("ts_monosyllable_frac", monosyllable_frac, np.float32),
    ("ts_flesch_reading_ease", flesch_reading_ease, np.float32),
    ("ts_flesch_kincaid_grade", flesch_kincaid_grade, np.float32),
    ("ts_gunning_fog", gunning_fog, np.float32),
    ("ts_smog_index", smog_index, np.float32),
    ("ts_automated_readability_index", automated_readability_index, np.float32),
    ("ts_coleman_liau_index", coleman_liau_index, np.float32),
    ("ts_linsear_write_formula", linsear_write_formula, np.float32),
    ("ts_dale_chall_readability_score", dale_chall_readability_score, np.float32),
    ("ts_difficult_words", difficult_words, np.float32),
    ("ts_spache_readability", spache_readability, np.float32),
    ("ts_mcalpine_eflaw", mcalpine_eflaw, np.float32),
]
for col, fn, dtype in stage1:
    print(col)
    df[col] = df.progress_apply(fn, axis=1)
    df[col] = df[col].astype(dtype)
for col, fn, dtype in stage2:
    print(col)
    df[col] = df.progress_apply(fn, axis=1)
    df[col] = df[col].astype(dtype)

ts_syllable_count


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 39120/39120 [00:07<00:00, 5491.23it/s]


ts_lexicon_count


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 39120/39120 [00:01<00:00, 33798.17it/s]


ts_sentence_count


...

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 39120/39120 [00:06<00:00, 5970.27it/s]


ts_spache_readability


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 39120/39120 [00:09<00:00, 3924.14it/s]


ts_mcalpine_eflaw


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 39120/39120 [00:04<00:00, 9739.43it/s]


# VAD Features

In [6]:
vdf = pd.read_csv("input/NRC-VAD-Lexicon/NRC-VAD-Lexicon.txt", header=0, names=["term", "valence", "arousal", "dominance"], 
                 sep="\t", engine="c", low_memory=False)
cols = ["valence", "arousal", "dominance"]
vdf[cols] = vdf[cols].astype(np.float32)
vdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19970 entries, 0 to 19969
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   term       19969 non-null  object 
 1   valence    19970 non-null  float32
 2   arousal    19970 non-null  float32
 3   dominance  19970 non-null  float32
dtypes: float32(3), object(1)
memory usage: 390.2+ KB


In [7]:
vad: Dict[str, Tuple] = {}
for t in tqdm(vdf.itertuples()):
    v = getattr(t, "valence")
    a = getattr(t, "arousal")
    d = getattr(t, "dominance")
    t = str(getattr(t, "term")).strip().lower()
    vad[t] = (v, a, d)

19970it [00:00, 1441358.94it/s]


In [8]:
cols = ["va_valence", "va_arousal", "va_dominance"]
rows = []
for text in tqdm(df["text_bow"]):
    vas, ars, dos = [], [], []
    tokens = text.split()
    for t in tokens:
        if t in vad:
            v, a, d = vad[t]
            vas.append(v)
            ars.append(a)
            dos.append(d)
    if len(vas) == 0:
        vas = [-1]
    if len(ars) == 0:
        ars = [-1]
    if len(dos)==0:
        dos = [-1]
    rows.append([np.mean(vas), np.mean(ars), np.mean(dos)])
df[cols] = rows
df[cols] = df[cols].astype(np.float32)
del rows, vdf, vad
gc.collect()

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 39120/39120 [00:02<00:00, 15048.38it/s]


32

# Review Data

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39120 entries, 0 to 39119
Data columns (total 41 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   essay_id                         39120 non-null  int32  
 1   generated                        39120 non-null  int8   
 2   source                           39120 non-null  object 
 3   prompt                           39120 non-null  object 
 4   text                             39120 non-null  object 
 5   text_bsc                         39120 non-null  object 
 6   text_bow                         39120 non-null  object 
 7   text_bow_len                     39120 non-null  int16  
 8   prompt_bsc                       39120 non-null  object 
 9   prompt_bow                       39120 non-null  object 
 10  prompt_bow_len                   39120 non-null  int16  
 11  white_sim                        39120 non-null  float32
 12  ch_len            

In [10]:
df.describe(percentiles=percentiles)

,essay_id,generated,text_bow_len,prompt_bow_len,white_sim,ch_len,ch_digit_frac,ch_letter_frac,ch_space_frac,ch_punc_frac,ch_upper_frac,ch_repeat_char_frac,ts_syllable_count,ts_lexicon_count,ts_sentence_count,ts_syllables_per_word,ts_syllables_per_sent,ts_words_per_sent,ts_polysyllable_frac,ts_monosyllable_frac,ts_flesch_reading_ease,ts_flesch_kincaid_grade,ts_gunning_fog,ts_smog_index,ts_automated_readability_index,ts_coleman_liau_index,ts_linsear_write_formula,ts_dale_chall_readability_score,ts_difficult_words,ts_spache_readability,ts_mcalpine_eflaw,va_valence,va_arousal,va_dominance
count,39120.000000,39120.000000,39120.000000,39120.000000,39120.000000,39120.000000,39120.000000,39120.000000,39120.000000,39120.000000,39120.000000,39120.000000,39120.000000,39120.000000,39120.000000,39120.000000,39120.000000,39120.000000,39120.000000,39120.000000,39120.000000,39120.000000,39120.000000,39120.000000,39120.000000,39120.000000,39120.000000,39120.000000,39120.000000,39120.000000,39120.000000,39120.000000,39120.000000,39120.000000
mean,19559.500000,0.247648,2210.626457,86.428119,0.842840,2245.470475,0.001567,0.799651,0.177732,0.021049,0.015692,0.017040,558.200665,396.847418,20.204729,1.401912,27.820436,19.868870,0.093032,0.719397,66.110062,9.369719,10.741475,10.709474,11.205893,8.946010,11.090199,7.269699,46.534740,4.901895,30.117283,0.646485,0.441732,0.562983
std,11293.115602,0.431652,999.526152,160.514247,0.169336,1015.203583,0.003022,0.015567,0.013099,0.008524,0.016748,0.005079,252.996314,175.796262,9.091115,0.140349,11.843336,9.132470,0.047810,0.073407,18.065680,5.668148,5.740438,2.265286,7.158312,2.464176,5.089449,1.200200,26.892366,2.019863,20.681490,0.037785,0.029359,0.036964
min,0.000000,0.000000,773.000000,2.000000,0.169103,773.000000,0.000000,0.593537,0.130141,0.000000,0.000000,0.000890,177.000000,113.000000,1.000000,1.044855,8.909091,7.148936,0.000000,0.426230,-628.880005,0.600000,3.300000,0.000000,0.400000,1.190000,2.600000,0.880000,2.000000,2.260000,9.800000,0.468342,0.284773,0.417888
1%,391.190000,0.000000,836.000000,2.000000,0.402919,848.190000,0.000000,0.759462,0.147296,0.007134,0.002219,0.007054,210.000000,153.000000,5.000000,1.163881,13.346154,10.322533,0.018100,0.533666,26.610001,3.400000,5.430000,6.200000,4.100000,4.280000,4.500000,5.420000,9.000000,3.010000,15.200000,0.553685,0.361721,0.489629
5%,1955.950000,0.000000,979.000000,2.000000,0.482188,993.000000,0.000000,0.773989,0.154505,0.010784,0.006226,0.009483,246.000000,180.000000,7.000000,1.214286,16.333334,12.392858,0.032652,0.582580,41.090000,4.800000,6.550000,7.500000,5.600000,5.440000,5.666667,5.830000,14.000000,3.410000,18.200001,0.584412,0.395472,0.508731
10%,3911.900000,0.000000,1124.000000,2.000000,0.544123,1141.000000,0.000000,0.780496,0.159614,0.012698,0.007449,0.010962,283.000000,206.900000,9.000000,1.244184,18.200001,13.583333,0.041667,0.617068,46.980000,5.500000,7.220000,8.100000,6.500000,6.040000,6.428571,6.050000,18.000000,3.650000,20.000000,0.599365,0.407844,0.518917
20%,7823.800000,0.000000,1379.000000,2.000000,0.679715,1401.000000,0.000000,0.787456,0.166775,0.015111,0.008911,0.012834,346.000000,252.000000,12.000000,1.285714,20.692308,15.187500,0.053991,0.659148,55.130001,6.600000,8.110000,8.800000,7.800000,6.860000,7.428571,6.360000,24.000000,3.970000,22.299999,0.615777,0.419550,0.531255
30%,11735.700000,0.000000,1596.000000,2.000000,0.838542,1622.000000,0.000000,0.792037,0.171388,0.016785,0.010279,0.014262,399.000000,292.000000,15.000000,1.317388,22.756558,16.384615,0.064220,0.686013,60.650002,7.400000,8.840000,9.500000,8.800000,7.480000,8.166667,6.620000,29.000000,4.240000,24.100000,0.627058,0.427742,0.541159
40%,15647.600000,0.000000,1804.000000,2.000000,0.879516,1834.000000,0.000000,0.795918,0.175230,0.018223,0.011808,0.015480,453.000000,328.000000,17.000000,1.346844,24.590910,17.461538,0.073684,0.707559,64.750000,8.100000,9.480000,10.000000,9.600000,8.060000,8.833333,6.880000,35.000000,4.470000,25.700001,0.636905,0.434588,0.550320


In [11]:
%%time
df.to_parquet(f"output/features.parquet", index=False)
assert df.notna().all(axis=None)

CPU times: user 553 ms, sys: 54 ms, total: 607 ms
Wall time: 606 ms


In [12]:
tim.stop()
print(f"Total time taken {str(tim.elapsed)}")

Total time taken 0:02:06.435577
